<h1>Análisis Exploratorio de Datos 2</h1>



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

color = sns.color_palette()

In [4]:
train_df = pd.read_hdf('../data/train_final.h5')
val_df = pd.read_hdf('../data/validacion_final.h5')

In [7]:
# ordenación por usuario 
train_df.sort_values(["ID"], inplace=True, ascending=True)

In [ ]:
#train_df.describe()
#train_df.groupby("ID_Customer").describe()

In [8]:
# numero de clientes en el entrenamiento
print('Número de clientes diferentes en train: {}'.format(len(train_df.ID.unique())))
print('Número de clientes diferentes en la validación: {}'.format(len(val_df.ID.unique())))

Número de clientes diferentes en train: 602873
Número de clientes diferentes en la validación: 676370


Podemos observar que serán 602873 los usuarios con los que entrenaremos, aunque realmente partíamos de 676370 hemos despreciado todos aquellos que solo aparecían una vez para entrenar.

In [11]:
# Número de productos diferentes
print('Número de registros: {}, Número de variables: {}'.format(*train_df.shape))
print('Número de registros: {}, Número de variables: {}'.format(*val_df.shape))

Número de registros: 2674231, Número de variables: 101
Número de registros: 676370, Número de variables: 101


Ya sabemos por el EDA 1 que hay 6 productos (601, 301, 201, 9991, 2302 y 9993) que se venden muchísimo. Uno de los productos, el 601, se vende tanto que un total de 561013 usuarios de 602873 lo tienen. Lo cual supone un 93.06% de los clientes que analizamos.

<h4>Buscando ruido</h4>
<p>Comprobamos cuántos 'missing values' hay en los datos:</p>

In [13]:
train_df.isnull().values.ravel().sum()

0

Podemos concluir que no hay ningún 'missing value' en todo el dataset

A simple vista parece que los valores de Socio_Demo nunca varían para un mismo usuario:

Asegurémonos de ello:

In [19]:
for column in train_df.columns[1:6]:
    groups = train_df[["ID", column]].groupby(["ID"])
    changes = groups.apply(lambda x: len(x[column].unique())) > 1
    if any(changes):
        print("Existen cambios en {}").format(str(column))
    else:
        print("No hay ningún cambio en {}".format(str(column)))

No hay ningún cambio en SD1
No hay ningún cambio en SD2
No hay ningún cambio en SD3
No hay ningún cambio en SD4
No hay ningún cambio en SD5


<p>Estas columnas de Socio_Demo provocan un extra de información inútil en nuestro dataset.</p>
<p>Nos planteamos entonces usar un solo registro para cada cliente, alterando las columnas para tener una columna por cada producto y rellenarla con valores booleanos</p>